# Notebook for scraping Toronto neighbourhood data from Wikipedia

Import necessary libraries:

In [22]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Fetch the html from Wikipedia and parse it into BS4 object

In [8]:
wiki_html = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(wiki_html,'lxml')

Iterate through the data structure and build a pandas dataframe
Deal with 'Not assigned' boroughs and collapse neighbourhoods with the same post code during the dataframe building

In [70]:
df_table=pd.DataFrame(columns=['PostalCode','Borough','Neighbourhood'])

table = soup.find('table',{'class':'wikitable sortable'})
trows = table.findAll('tr')
for trow in trows:
    g=trow.stripped_strings
    c0=next(g)
    c1=next(g)
    c2=next(g)
    if c2=='Not assigned':
        c2=c1
    if c1 != 'Not assigned':    
        if df_table.shape[0]>0 and c0==df_table.iloc[-1]['PostalCode']:
            df_table.iloc[-1]['Neighbourhood']=df_table.iloc[-1]['Neighbourhood']+", "+c2
        else:                
            df_table=df_table.append({'PostalCode' : c0 , 'Borough' : c1, 'Neighbourhood':c2} , ignore_index=True)

# drop first row(column titles) from the dataframe
df_table.drop(df_table.index[[0]],inplace=True)
df_table.reset_index(inplace=True,drop=True)
df_table.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


Print out the shape of constructed table

In [72]:
df_table.shape       

(103, 3)